### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/34566/Boruto__Naruto_Next_Generations/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,13):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/34566/Boruto__Naruto_Next_Generations/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  240 non-null    object
dtypes: object(1)
memory usage: 2.0+ KB
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n1\n\n\nStory\n1\n\n\nAnimatio...
1,\n\n\n\nOverall\n1\n\n\nStory\n1\n\n\nAnimatio...
2,\n\n\n\nOverall\n1\n\n\nStory\n1\n\n\nAnimatio...
3,\n\n\n\nOverall\n3\n\n\nStory\n2\n\n\nAnimatio...
4,\n\n\n\nOverall\n8\n\n\nStory\n9\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

 Overall 1 Story 1 Animation 7 Sound 8 Character 1 Enjoyment 1 Yikes, just yikes. Because of my love for Naruto I really tried to delude myself into thinking this show wasn't that bad. But really, what else is there to say about Boruto other than it's an utter disgrace. Like most others I wanted to watch this show because I'm sentimental when it comes to Naruto, making it that much worse of a betrayal. "Wow brotato chip you're sounding pretty salty." Ding Ding Ding we have a winner! To say the show reeks of a money grab is an understatement. Like driving without a steering wheel, the show has no direction. To begin the show they try to reel you in with a spoiler that is so ludicrous that it gets to the point where it almost tarnishes the image of Naruto. Story: Not much to say, there is no plot. You really can't analyze any kind of story here because it just doesn't exist. We thought the shinobi world would be slower after Naruto but nobody even has a goal or purpose in life anymore. T

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'yikes', 'just', 'yikes', 'because', 'of', 'my', 'love', 'for', 'naruto', 'really', 'tried', 'to', 'delude', 'myself', 'into', 'thinking', 'this', 'show', 'wasn', 'that', 'bad', 'but', 'really', 'what', 'else', 'is', 'there', 'to', 'say', 'about', 'boruto', 'other', 'than', 'it', 'an', 'utter', 'disgrace', 'like', 'most', 'others', 'wanted', 'to', 'watch', 'this', 'show', 'because', 'sentimental', 'when', 'it', 'comes', 'to', 'naruto', 'making', 'it', 'that', 'much', 'worse', 'of', 'betrayal', 'wow', 'brotato', 'chip', 'you', 're', 'sounding', 'pretty', 'salty', 'ding', 'ding', 'ding', 'we', 'have', 'winner', 'to', 'say', 'the', 'show', 'reeks', 'of', 'money', 'grab', 'is', 'an', 'understatement', 'like', 'driving', 'without', 'steering', 'wheel', 'the', 'show', 'has', 'no', 'direction', 'to', 'begin', 'the', 'show', 'they', 'try', 'to', 'reel', 'you', 'in', 'with', 'spoiler', 'that', 'is', 'so', 'ludicrous', 'that', 

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'hello', 'anime', 'lovers', 'no', 'spoilers', 'short', 'the', 'first', 'episodes', 'are', 'fillers', 'some', 'are', 'very', 'good', 'and', 'enjoyable', 'funny', 'others', 'are', 'ok', 'and', 'there', 'are', 'some', 'meh', 'still', 'though', 'despite', 'the', 'fact', 'that', 'are', 'fillers', 'still', 'you', 'don', 'want', 'to', 'miss', 'some', 'important', 'arcs', 'of', 'this', 'episodes', 'episode', 'is', 'like', 'the', 'worst', 'episode', 'ever', 'take', 'deep', 'breath', 'on', 'this', 'one', 'episode', 'is', 'recap', 'of', 'what', 'happend', 'so', 'far', 'reminding', 'you', 'the', 'progression', 'of', 'most', 'of', 'the', 'characters', 'from', 'episode', 'starts', 'the', 'real', 'thing', 'and', 'particularly', 'episodes', 'later', 'starts', 'the', 'momoshiki', 'arc', 'first', 'manga', 'arc', 'this', 'arc', 'also', 'is', 'on', 'the', 'boruto', 'naruto', 'the', 'movie', 'but', 'limitless', 'scenes', 'or', 'dialogues'

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'yike', 'yike', 'love', 'naruto', 'really', 'try', 'delude', 'thinking', 'show', 'bad', 'really', 'else', 'say', 'boruto', 'utter', 'disgrace', 'other', 'want', 'watch', 'show', 'sentimental', 'come', 'naruto', 'make', 'much', 'bad', 'betrayal', 'brotato', 'chip', 'sound', 'pretty', 'salty', 'ding', 'ding', 'ding', 'winner', 'say', 'show', 'reeks', 'money', 'grab', 'understatement', 'drive', 'steer', 'wheel', 'show', 'direction', 'begin', 'show', 'try', 'reel', 'spoiler', 'ludicrous', 'get', 'point', 'almost', 'tarnish', 'image', 'naruto', 'story', 'much', 'say', 'plot', 'really', 'analyze', 'kind', 'story', 'exist', 'think', 'shinobi', 'world', 'slower', 'naruto', 'even', 'goal', 'purpose', 'life', 'anymore', 'drive', 'force', 'individual', 'village', 'nation', 'character', 'show', 'goal', 'purpose', 'sarada', 'uchiha', 'character', 'whole', 'idea', 'hokage', 'stamp', 'paper', 'funny', 'naruto', 'take', 'way', 'far',

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 5), (17, 1), (18, 1), (19, 1), (20, 6), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 2), (66, 1), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 2), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 1), (109, 1), (110, 2),

In [21]:
# Using LDA to develop a topic model with 3 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 3 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.033*"character" + 0.028*"story" + 0.020*"naruto" + 0.019*"anime" + 0.014*"sound" + 0.013*"boruto" + 0.012*"overall" + 0.012*"see" + 0.012*"enjoyment" + 0.011*"series"'), (1, '0.033*"naruto" + 0.028*"boruto" + 0.026*"character" + 0.017*"series" + 0.013*"show" + 0.011*"see" + 0.009*"much" + 0.009*"story" + 0.008*"think" + 0.008*"time"'), (2, '0.030*"naruto" + 0.027*"character" + 0.027*"boruto" + 0.021*"episode" + 0.019*"story" + 0.018*"anime" + 0.017*"good" + 0.015*"watch" + 0.015*"get" + 0.013*"animation"'), (3, '0.020*"anime" + 0.008*"wait" + 0.008*"naruto" + 0.006*"battle" + 0.006*"break" + 0.006*"pain" + 0.006*"shallow" + 0.006*"sasuke" + 0.006*"release" + 0.006*"original"'), (4, '0.003*"fantasy" + 0.003*"becouse" + 0.003*"tactic" + 0.003*"construction" + 0.003*"measure" + 0.002*"rule" + 0.002*"disneyish" + 0.002*"closed" + 0.002*"disorder" + 0.002*"children"'), (5, '0.007*"husk" + 0.002*"baruto" + 0.001*"doll" + 0.001*"obv" + 0.000*"mal" + 0.000*"endless" + 0.000*"sadly" + 0

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.249506  0.004244       1        1  58.152795
1      0.199994  0.037888       2        1  27.032205
0      0.173194 -0.050416       3        1   9.577925
6     -0.116322  0.131561       4        1   2.601835
3     -0.124944 -0.131932       5        1   2.256486
4     -0.137607  0.001942       6        1   0.343721
5     -0.122646  0.003071       7        1   0.025736
7     -0.121175  0.003643       8        1   0.009297, topic_info=          Term         Freq        Total Category  logprob  loglift
5        anime   506.000000   506.000000  Default   30.000  30.0000
20   character   984.000000   984.000000  Default   29.000  29.0000
168      story   623.000000   623.000000  Default   28.000  28.0000
121     naruto  1075.000000  1075.000000  Default   27.000  27.0000
353     series   310.000000   310.000000  Default   26.000  26.0000
..         ...          ...          ...      ...      ...      ...
47   enjoyment     0.000874   306.001542   Topic8   -8.306  -3.4827
194      world     0.000874    84.982773   Topic8   -8.306  -2.2015
162      sound     0.000874   342.336005   Topic8   -8.306  -3.5949
221       good     0.000874   424.910316   Topic8   -8.306  -3.8110
100       know     0.000874   131.573464   Topic8   -8.306  -2.6387

[539 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
606       2  0.279178   abandon
606       4  0.837534   abandon
607       2  0.957689   ability
2038      2  0.925234  academia
1102      1  0.951431   academy
...     ...       ...       ...
194       2  0.494218     world
194       4  0.035301     world
390       1  0.909024     worth
390       2  0.066514     worth
390       3  0.022171     worth

[616 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 7, 4, 5, 6, 8])

In [23]:
# Save the data frame as a CSV file
# job_datas.to_csv(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Programming Assignment 2\Chan_Matthew_INST447_PA2.csv')

In [24]:
# Week 9 Topic Modelling
# ner_corpus = ' '.join(ner_df['Name'])
# corp = open(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Week 9\corpus.txt',"w")
# corp.writelines(ner_corpus)
# corp.close()